In [22]:
import pandas as pd

In [23]:
users = pd.read_pickle('data/02_intermediate/users.pkl')

In [24]:
deprecated = ["utc_offset", "time_zone", "lang", "geo_enabled", "following", "follow_request_sent", "has_extended_profile", "notifications", "profile_location", "contributors_enabled", "profile_image_url", "profile_background_color", "profile_background_image_url", "profile_background_image_url_https", "profile_background_tile", "profile_link_color", "profile_sidebar_border_color", "profile_sidebar_fill_color", "profile_text_color", "profile_use_background_image", "is_translator", "is_translation_enabled", "translator_type"]
users.drop(columns=deprecated, inplace=True)

In [25]:
def extract_urls(entities):
    profile_url = None
    if "url" in entities:
        url = entities["url"]["urls"][0]
        profile_url = url["expanded_url"] if url["expanded_url"] else url["url"] 
    description_urls = [url["expanded_url"] for url in entities["description"]["urls"] if url]
    
    return pd.Series([profile_url, description_urls])

In [26]:
#extracting date of last tweet
users["last_status_date"] = pd.to_datetime(users["status"].apply(lambda x: None if pd.isnull(x) else x["created_at"]))

In [28]:
#extracting expanded profile url and urls from the description 
users[["profile_url", "description_urls"]] = users["entities"].apply(extract_urls)

In [29]:
users.drop(['idx', 'status', 'entities', 'url'], axis=1, inplace=True)

In [31]:
#removing old accounts with no tweets publicly available
users = users[~users["last_status_date"].isnull()]

In [32]:
users.to_pickle('data/03_primary/users.pkl')